In [ ]:
import cv2
import numpy as np
from jetbot import Camera, bgr8_to_jpeg, Robot
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import time
from datetime import datetime

# Initialize the camera with a specific width and height
camera = Camera.instance(width=224, height=224)

# Create image widgets to display the camera feed and green mask
image_widget = widgets.Image(format='jpeg', width=224, height=224)
green_mask_widget = widgets.Image(format='jpeg', width=224, height=224)
green_area_widget = widgets.IntText(
    value=0,
    description='Green Area:',
    disabled=True,
    layout=widgets.Layout(width='200px')
)

# Initialize the robot
robot = Robot()

# Global variables for recovery mechanism and cooldown timer
last_direction = None  # Last known direction ('left' or 'right')
last_green_detection_time = 0  # Timestamp of the last green detection
COOLDOWN_TIME = 3.8   # Cooldown period in seconds


def process_image(camera_value, crop_ratio_vertical=0, crop_ratio_horizontal=0):
    global last_direction, last_green_detection_time
    # Define speeds
    MAX_LINEAR_SPEED = -0.11
    MIN_LINEAR_SPEED = -0.11
    ANGULAR_SPEED = 0.045
    STRAFE_SPEED = -0.15
    UTURN_SPEED = 0.15
    # Get the current time
    current_time = time.time()

    # Get the dimensions of the original image
    height, width, _ = camera_value.shape

    # Calculate cropping boundaries based on crop_ratio
    crop_top = int(height * crop_ratio_vertical / 2)
    crop_bottom = height - crop_top
    crop_left = int(width * crop_ratio_horizontal / 2)
    crop_right = width - crop_left

    # Crop the BGR image based on specified vertical and horizontal ratios
    cropped_bgr = camera_value[crop_top:crop_bottom, crop_left:crop_right]

    # Enhance the red channel to reduce noise
    b, g, r = cv2.split(cropped_bgr)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))  # CLAHE for local contrast enhancement
    r_eq = clahe.apply(r)  # Apply CLAHE to the red channel
    img_rgb_eq = cv2.merge([b, g, r_eq])

    # Convert to HSV for color-based segmentation
    hsv = cv2.cvtColor(img_rgb_eq, cv2.COLOR_BGR2HSV)
    lower_green = np.array([80, 90, 85])   # Adjusted thresholds for green detection
    upper_green = np.array([110, 200, 160])
    green_mask = cv2.inRange(hsv, lower_green, upper_green)

    # Apply morphological operations to clean up the green mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_CLOSE, kernel)  # Close gaps in the mask
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)   # Remove small noise

    # Focus on the middle part of the camera for green area calculation
    middle_top = int(height * 0.3)  # Top boundary for middle part
    middle_bottom = int(height )  # Bottom boundary for middle part
    middle_mask = green_mask[middle_top:middle_bottom, :]  # Extract middle part

    # Calculate the area of the green mask in the middle part using contours
    TURN_SIZE_THRESHOLD = 160
    green_detected = False
    green_centers = []

    contours, _ = cv2.findContours(middle_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    green_area = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 100:  # Ignore small noisy regions
            green_area += area
            M = cv2.moments(contour)
            if M['m00'] > 0:
#                 green_x = int(M['m10'] / M['m00'])  # X-coordinate of the centroid
#                 green_y = int(M['m01'] / M['m00'])  # Y-coordinate of the centroid
#                 green_y_array.append((green_y))  # Store as a tuple (X, Y)
                green_centers.append(int(M['m10'] / M['m00']))  # Calculate the centroid X position
            if green_area >= TURN_SIZE_THRESHOLD:
                green_detected = True
#                 break;

    # Update the green area widget
    green_area_widget.value = green_area
    print(f"Green Area in middle part: {green_area}")

    # Red Line Detection Logic
    lower_red1 = np.array([0, 120, 70])     # Lower range for red
    upper_red1 = np.array([10, 255, 255])  # Upper range for red
    lower_red2 = np.array([170, 120, 70])   # Second range for red (hue wrap-around)
    upper_red2 = np.array([179, 255, 255])

    # Create masks for red regions in both ranges
    red_mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    red_mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(red_mask1, red_mask2)  # Combine both red masks

    # Focus on the lower 15% of the screen for red detection
    lower_red_top = int(height * 0.85)  # Start of the lower 15% region
    lower_red_mask = red_mask[lower_red_top:, :]  # Extract the lower 15% part

    # Find contours in the red mask
    red_detected = False
    contours, _ = cv2.findContours(lower_red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area_red = cv2.contourArea(contour)
        if area_red > 1000:  # Threshold for significant red regions
            red_detected = True
            print("Red line detected. Stopping the robot.")
            robot.stop()  # Stop the robot
            return  # Exit the function


        # Line Detection Logic
    # Convert the cropped image to grayscale
       # Convert the cropped image to grayscale
    gray = cv2.cvtColor(cropped_bgr, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Kernel size (5, 5) and sigma = 0

    # Get the dimensions of the grayscale image
    height, width = gray_blurred.shape

    # Define central and edge regions
    edge_width = 65  # Width of the edge region
    center_region = gray_blurred[edge_width:height-edge_width, edge_width:width-edge_width]  # Center part
    top_edge = gray_blurred[:edge_width, :]  # Top edge region
    bottom_edge = gray_blurred[-edge_width:, :]  # Bottom edge region
    left_edge = gray_blurred[:, :edge_width]  # Left edge region
    right_edge = gray_blurred[:, -edge_width:]  # Right edge region

    # Apply normal processing to the center
    _, center_thresh = cv2.threshold(center_region, 100, 255, cv2.THRESH_BINARY_INV)

    # Apply black filtering to the edges
    lower_black = 0  # Define the lower threshold for black (pure black)
    upper_black = 55  # Define the upper threshold for black (adjust as needed)
    _, top_thresh = cv2.threshold(top_edge, upper_black, 255, cv2.THRESH_BINARY_INV)
    _, bottom_thresh = cv2.threshold(bottom_edge, upper_black, 255, cv2.THRESH_BINARY_INV)
    _, left_thresh = cv2.threshold(left_edge, upper_black, 255, cv2.THRESH_BINARY_INV)
    _, right_thresh = cv2.threshold(right_edge, upper_black, 255, cv2.THRESH_BINARY_INV)

    # Combine the processed regions into a single image
    thresh = np.zeros_like(gray_blurred)  # Create a blank image
    thresh[edge_width:height-edge_width, edge_width:width-edge_width] = center_thresh  # Fill the center
    thresh[:edge_width, :] = top_thresh  # Fill the top edge
    thresh[-edge_width:, :] = bottom_thresh  # Fill the bottom edge
    thresh[:, :edge_width] = left_thresh  # Fill the left edge
    thresh[:, -edge_width:] = right_thresh  # Fill the right edge

    # Get the dimensions of the combined binary image
    cropped_height, cropped_width = thresh.shape




    # Line following ROI
    roi = thresh[int(cropped_height * 0.4):, :]  # Bottom half for line detection
    moments = cv2.moments(roi)
    line_detected = moments['m00'] > 0
    cx = int(moments['m10'] / moments['m00']) if line_detected else cropped_width // 2



    # Invalid turn detection and U-turn logic update
    if green_detected and current_time - last_green_detection_time > COOLDOWN_TIME:
        invalid_turn = False

        for center in green_centers:
            # Define area around green marker
            green_x = int(center)
            crop_start_x = max(green_x - 25, 0)
            crop_end_x = min(green_x + 25, cropped_width)
            crop_start_y = middle_top - 200
            crop_end_y = crop_start_y + 400
            cropped_area = thresh[crop_start_y:crop_end_y, crop_start_x:crop_end_x]

            white_area = np.sum(cropped_area == 255)
            total_area = cropped_area.shape[0] * cropped_area.shape[1]
            white_percentage = (white_area / total_area) * 100 if total_area > 0 else 0
            print(f"White Percentage: {white_percentage:.2f}%")

            if white_percentage > 45:
                print("Invalid turn detected near green marker. Skipping turn.")
                last_green_detection_time = current_time
                invalid_turn = True
                break


        left_green = any(center < cx for center in green_centers)  # Green on the left of the line
        right_green = any(center > cx for center in green_centers)  # Green on the right of the line
        # New behavior: move straight towards the green marker
        green_x = np.mean(green_centers)
        error = green_x - cropped_width // 2
        print(f"Green detected. Center X: {green_x}, Error: {error}")

#         if abs(error) < 14:
#             print("Moving straight towards the green marker.")
# #             robot.set_motors(-MIN_LINEAR_SPEED, -MIN_LINEAR_SPEED)
#         elif error > 0:
#             print("Adjusting right to align with the green marker.")
# #             robot.set_motors(-MIN_LINEAR_SPEED + ANGULAR_SPEED, -MIN_LINEAR_SPEED - ANGULAR_SPEED)
#         else:
#             print("Adjusting left to align with the green marker.")
# #             robot.set_motors(-MIN_LINEAR_SPEED - ANGULAR_SPEED, -MIN_LINEAR_SPEED + ANGULAR_SPEED)

#         time.sleep(0.3)  # Move straight for 1 second
#         robot.stop()
#         last_green_detection_time = current_time


        # Existing turn logic
        if invalid_turn:
            last_green_detection_time = current_time
            robot.set_motors(-MIN_LINEAR_SPEED, -MIN_LINEAR_SPEED)  # U-turn motion
            time.sleep(1)  # Pause for U-turn
            robot.stop()
            time.sleep(1.7)
            robot.stop()


        elif left_green and right_green:  # Green detected on both sides of the line
            print("Green detected on both sides of the line. Performing U-turn.")
            last_green_detection_time = current_time
            robot.set_motors(-MIN_LINEAR_SPEED, -MIN_LINEAR_SPEED)  # U-turn motion
            time.sleep(1.5)
            robot.stop()

            robot.set_motors(0.25, -0.25)  # U-turn motion
            time.sleep(0.625)  # Pause for U-turn
            robot.stop()
        elif left_green:  # Green on the left
            print("Green is to the left of the line. Turning left.")
            last_green_detection_time = current_time
            robot.set_motors(-MIN_LINEAR_SPEED, -MIN_LINEAR_SPEED)  # U-turn motion
            time.sleep(1.5)
            robot.set_motors(-UTURN_SPEED,UTURN_SPEED)  # U-turn motion
            time.sleep(0.55)
            robot.stop()
            time.sleep(1)
            robot.stop()

        elif right_green:  # Green on the right
            print("Green is to the left of the line. Turning left.")
            last_green_detection_time = current_time
            robot.set_motors(-MIN_LINEAR_SPEED, -MIN_LINEAR_SPEED)  # U-turn motion
            time.sleep(1.5)
            robot.set_motors(UTURN_SPEED,-UTURN_SPEED)  # U-turn motion
            time.sleep(0.55)
            robot.stop()
            time.sleep(1)
            robot.stop()


    # Obstacle Detection Logic
    obstacle_roi = thresh[:int(cropped_height * 0.5), :]  # Top half for obstacle detection
    obstacle_contours, _ = cv2.findContours(obstacle_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    obstacle_detected = False
    for contour in obstacle_contours:
        area = cv2.contourArea(contour)
        if area > 4700:  # Threshold for detecting obstacles
            obstacle_detected = True
            print("Obstacle detected!")
            break

    # Default behavior if no green or obstacle
    if obstacle_detected:
        print("Stopping due to obstacle!")
        robot.stop()
    else:
        # Line following logic
        if line_detected:
            error = cx - cropped_width // 2
            print(f"Line detected. CX: {cx}, Error: {error}")
            if abs(error) < 10:
                robot.set_motors(-MIN_LINEAR_SPEED, -MIN_LINEAR_SPEED)
            elif error > 0:
                last_direction = 'right'
                robot.set_motors(-MIN_LINEAR_SPEED + ANGULAR_SPEED, -MIN_LINEAR_SPEED - ANGULAR_SPEED)
            else:
                last_direction = 'left'
                robot.set_motors(-MIN_LINEAR_SPEED - ANGULAR_SPEED, -MIN_LINEAR_SPEED + ANGULAR_SPEED)

        else:
            robot.set_motors(-MAX_LINEAR_SPEED, -MAX_LINEAR_SPEED)  # Default forward motion

    # Update green mask widget with the green mask
    green_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(green_mask, cv2.COLOR_GRAY2BGR))

    return bgr8_to_jpeg(cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR))

# Link the processed image to the image widget
camera_link = traitlets.dlink(
    (camera, 'value'),
    (image_widget, 'value'),
    transform=lambda img: process_image(img, crop_ratio_vertical=0, crop_ratio_horizontal=0)
)

# Display the widgets in the notebook
display(widgets.HBox([image_widget, green_mask_widget, green_area_widget]))